# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [33]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

# Access maps with unique API key
gkey = ""
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:

# open csv file from Part I
weather_data_path = "../output_data/weather_data.csv"
weather_data = pd.read_csv(weather_data_path)
weather_data = weather_data.rename(columns = {"temperature (F)" : "Temperature (C)",
                                            "Temp_max (F)" : "Max Temperature (C)"})
weather_data.head()

,city,latitutes,longtitudes,Temperature (C),Max Temperature (C),humidity,cloudiness,wind speed,date,country
0,ust-tsilma,65.4410,52.1498,-9.90,-9.90,87,100,5.76,1644434596,RU
1,yellowknife,62.4560,-114.3525,-23.71,-23.71,70,100,4.12,1644434475,CA
2,ribeira grande,38.5167,-28.7000,16.22,16.22,67,20,5.14,1644434597,PT
3,tiksi,71.6872,128.8694,-24.57,-24.57,93,100,5.79,1644434597,RU
4,kavieng,-2.5744,150.7967,27.19,27.19,80,88,3.01,1644434598,PG


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
# Create a list containing coordinates
humidity = weather_data["humidity"]

In [27]:
# Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

In [5]:
# Store latitude and longitude in locations
locations = weather_data[["latitutes", "longtitudes"]]

# Fill NaN values and convert to float
humidity = weather_data["humidity"].astype(float)

In [28]:
# Plot Heatmap
fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=20,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# condition to travel (based on my family's interests): temp <=25 and >=18 | zero cloudiness | humidity <=40
temp = weather_data["Max Temperature (C)"].astype(float)
new_df = weather_data.loc[(weather_data["Max Temperature (C)"]< 25) & (weather_data["Max Temperature (C)"]>=18) &
 (weather_data["humidity"]<40) & (weather_data["cloudiness"] == 0)]
clean_df = new_df.dropna()
us_cities = clean_df.loc[(clean_df["country"] == "US")]
us_cities.head()
#len(new_df) # the result show there are 13 cities that my family may plan to visit, but we just wanna travel within the US this year so,
# I choose to filter out cities in the US only; 

,city,latitutes,longtitudes,Temperature (C),Max Temperature (C),humidity,cloudiness,wind speed,date,country
275,san patricio,28.0170,-97.5169,20.40,23.05,33,0,1.17,1644434944,US
294,pineville,31.3224,-92.4343,18.76,19.37,31,0,6.69,1644434967,US
352,lawton,34.6087,-98.3903,19.80,20.60,13,0,4.02,1644435038,US
427,leon valley,29.4952,-98.6186,20.78,22.27,28,0,5.14,1644434923,US
453,stephenville,32.2207,-98.2023,18.85,21.05,16,0,3.60,1644435163,US


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = us_cities[["city","latitutes","longtitudes"]]
hotel_df.insert(1, "Hotel Name", " ", True)
hotel_df.insert(2, "Hotel Address", " ", True)
hotel_df.head()

,city,Hotel Name,Hotel Address,latitutes,longtitudes
275,san patricio,,,28.0170,-97.5169
294,pineville,,,31.3224,-92.4343
352,lawton,,,34.6087,-98.3903
427,leon valley,,,29.4952,-98.6186
453,stephenville,,,32.2207,-98.2023


In [10]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify the hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["latitutes"]
    lng = row["longtitudes"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    hotel_address = hotel_address.json()
    pprint(hotel_address)

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = hotel_address["results"][0]["vicinity"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        pass
print('----job done-----')

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 28.0285823,
                                        'lng': -97.52950510000001},
                           'viewport': {'northeast': {'lat': 28.02992117989272,
                                                      'lng': -97.52868277010728},
                                        'southwest': {'lat': 28.02722152010728,
                                                      'lng': -97.53138242989273}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Americas Best Value Inn Sinton',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 1294,
                          'html_attributions': ['<a '
               

/Users/ngocdiep/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


{'html_attributions': [],
 'next_page_token': 'Aap_uEAoh-Uiqjq10bIX8g7s1B1HnTjqjYOPiJB9Zc2gsbrz7QMMFnahdGKV5gqk3NL9FY-KnkXN_4bDTyzJIYrNULWvztyzbnssR1dCNxOm9h7AMWRiv9IsU_mSSO-UetO84iFK9Xyzzq_It8Ni1rdM9CcL3k1_3h5G18maKyjaRyT045akg-1LxbJ4MBSTaIOERbogHNnxks3RR6KgbxH-hBqgjfqPJTpmgjQu2kmQ98y7oKxkSGkpninpCs22CX5Uhh55JM3v-OXx8fngUFC6_6zOaFgJN1JKzGYcND9A5JvC2BgHVV6nrXqWHBiwKZrOv0U_eOXYb_c7eJS4zgiIgkEgEQqbX-aoNyMcoPXdkeHLkqLl4Q6QjaifupVjFf_MUJEo44GnndVgxOmufm2-pKeUaf6d0VCos5eMR4rtXtq_A0kW7pPRA5KsJFdMmQs',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 31.3562847,
                                        'lng': -92.42162669999999},
                           'viewport': {'northeast': {'lat': 31.35719877989272,
                                                      'lng': -92.41972237010728},
                                        'southwest': {'lat': 31.35449912010728,
                                                      'lng': -92.42242202989272}}}

In [18]:
hotel_df.head()


,city,Hotel Name,Hotel Address,latitutes,longtitudes
275,san patricio,Americas Best Value Inn Sinton,"8108 US-77, Sinton",28.0170,-97.5169
294,pineville,"Country Inn & Suites by Radisson, Pineville, LA","2727 Monroe Hwy #165, Pineville",31.3224,-92.4343
352,lawton,Hilton Garden Inn Lawton-Fort Sill,"135 NW 2nd St, Lawton",34.6087,-98.3903
427,leon valley,Hyatt Place San Antonio-Northwest/Medical Center,"4303 Hyatt Pl Dr, San Antonio",29.4952,-98.6186
453,stephenville,La Quinta Inn & Suites by Wyndham Stephenville,"105 Christy Plaza, Stephenville",32.2207,-98.2023


In [22]:
to_mark = hotel_df[['latitutes', 'longtitudes']]
to_mark

,latitutes,longtitudes
275,28.0170,-97.5169
294,31.3224,-92.4343
352,34.6087,-98.3903
427,29.4952,-98.6186
453,32.2207,-98.2023
549,36.6777,-121.6555
556,33.4251,-94.0477


In [29]:
# Create a map and set markers
marker_hotel = to_mark
hotel_name = hotel_df["Hotel Name"]

# Create a marker_layer using the hotel name and address to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_hotel,
    info_box_content=[f"Hotel: {hotel_name}" for name in hotel_name])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [3]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
